In [1]:
import pathlib
import pickle
import os
import pandas as pd
from helpers.tempcomp_helper import get_comp_factors
from helpers.tempcomp_helper import add_compensated_data_to_df
from helpers.tempcomp_helper import add_compensated_data_to_df_infrequent

In [8]:
test_date = "08_02_2022"
test_folder = "Day2_Dynamic1"
test_name = "normalized_WTRUN2_day2_dynamic1_2022-08-02_14-32-54-11_rtd-str"
# test_date = "08_01_2022"
# test_folder = "Day1_Training1"
# test_name = "normalized_WTRUN2_training_sweep1_2022-08-01_17-24-43-50_rtd-str"

cur_dir = os.getcwd()
par_dir = os.path.abspath(os.path.join(cur_dir, os.pardir))
up_dir = os.path.dirname(par_dir)
data_dir = os.path.join(up_dir, test_date+"_Tests", "testdata", test_folder)
test_csv = os.path.join(data_dir, test_name+".csv")

if "normalized" not in test_csv:
  _ = input("The .csv file is not normalized. Are you sure you'd like to continue?")

test_df = pd.read_csv(test_csv, header=1) #header=1 for normalized SGs/RTDs. header=0 for non-normalized

# 1) Get Comp Factors
The block below computes the compensation factors and exports them in .pickle format

In [3]:
if "heating_pretest" not in test_name:
  _ = input("""You're trying to set compensation factors from a 
            different test than the controlled pre-heating test. 
            This is very likely not what you should do! Are you sure?""")

sensor_id_list = [1, 2, 4, 5, 6]
unique_rtds = False #Set whether we'd like to find compensation factors using unique RTDs or shared RTDs
comp_factors = get_comp_factors(test_df, sensor_id_list, unique_rtds)

Uniform filter applied to RTD 6.
Uniform filter applied to RTD 6.


In [4]:
if not unique_rtds: #If we only have RTD 1 and RTD 6 available
  data_name = "compfactors_sharedRTDs.pickle"
else:
  data_name = "compfactors_uniqueRTDs.pickle"

with open(os.path.join(data_dir, data_name), 'wb') as f:
  pickle.dump(comp_factors, f)

# 2) Add Compensated Strains
The block below computes the compensated data and creates a new .csv file with them in it.

In [9]:
unique_rtds = True #Set whether we'd like to find compensation factors using unique RTDs or shared RTDs

if unique_rtds == False: #If we only have RTD 1 and RTD 6 available
  data_name = "compfactors_sharedRTDs.pickle"
else:
  data_name = "compfactors_uniqueRTDs.pickle"

with open(os.path.join(data_dir, data_name), 'rb') as f:
  comp_factors = pickle.load(f)

test_df = add_compensated_data_to_df_infrequent (test_df, comp_factors)
# test_df = add_compensated_data_to_df (test_df, comp_factors, unique_rtds)

compensated_csv = os.path.join(data_dir, "compensated_"+test_name+"_uniqueRTDs_infrequent.csv")
test_df.to_csv(compensated_csv, index=False)

Uniform filter applied to RTD 6.
